### Introduction

##### Purpose of the Notebook:

Scrape Wikipedia for the data of (mass) shootings in the USA.

### 1. Imports
##### 1.1 Import libraries and write settings.

In [1]:
import pandas as pd
import numpy as np

# Options for pandas
pd.options.display.max_columns = None
pd.options.display.max_rows = 30

import warnings
warnings.filterwarnings("ignore")

import requests
from bs4 import BeautifulSoup

import os
import time
import datetime

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_mass_shootings_in_the_United_States_in_2019'

In [3]:
headers = {'User-Agent': 
           'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}
pageTree = requests.get(url, headers=headers)
soup = BeautifulSoup(pageTree.content, 'html.parser')

Find the table that stores the data

In [4]:
tables = soup.find_all("tbody")
table = tables[0]

loop through the rows and write the column contents into our data list

In [5]:
data = []
rows = table.find_all('tr')

column_headers = rows[0].find_all('th')
col_names = [element.text.strip('\n') for element in column_headers]

for row in rows[1:]:
    cols = row.find_all('td')
    cols = [element.text for element in cols]
    data.append([element.strip('\n') for element in cols if element]) # Get rid of empty values

In [6]:
df = pd.DataFrame(data, columns = col_names)

##### Clean the DataFrame

In [7]:
df['dt'] = pd.to_datetime(df.Date, format = '%B %d, %Y')

df['Suspects Dead'] = 0
df['Suspects Dead'] = (df.loc[df['Dead']
                            .str.contains('n \d'), 'Dead']
                            .str.split('n', expand = True)
                            .iloc[:, 1]
                            .str.replace(']', '')
                            .str.strip())
df['Suspects Injured'] = 0
df['Suspects Injured'] = (df.loc[df['Injured']
                            .str.contains('n \d'), 'Injured']
                            .str.split('n', expand = True)
                            .iloc[:, 1]
                            .str.replace(']', '')
                            .str.strip())
                
df['Suspects Dead'] = df['Suspects Dead'].fillna(0)
df['Suspects Injured'] = df['Suspects Injured'].fillna(0)

df['Dead'] = df.Dead.str.replace('n \d', '', regex = True).str.replace('[]', '', regex = False)
df['Injured'] = df.Injured.str.replace('n \d', '', regex = True).str.replace('[]', '', regex = False)

df[['Dead', 'Injured', 'Total']] = df[['Dead', 'Injured', 'Total']].astype('int')
df['Year'] = df['dt'].apply(lambda x: x.year)
df['Month'] = df['dt'].apply(lambda x: x.month)
df['Day'] = df['dt'].apply(lambda x: x.day)

In [8]:
df['City'] = df.Location.apply(lambda x: x.split(',')[0])
df['State'] = df.Location.apply(lambda x: x.split(',')[-1])

In [9]:
df.head()

,Date,Location,Dead,Injured,Total,Description,dt,Suspects Dead,Suspects Injured,Year,Month,Day,City,State
0,"September 8, 2019","Sumter, South Carolina",2,3,5,Two men were killed and three other individual...,2019-09-08,0,0,2019,9,8,Sumter,South Carolina
1,"September 7, 2019","Whiteville, North Carolina",3,1,4,"A husband, wife and their five-year-old son we...",2019-09-07,0,0,2019,9,7,Whiteville,North Carolina
2,"September 4, 2019","Marrero, Louisiana",2,2,4,"A man and a seven-year-old girl were killed, a...",2019-09-04,0,0,2019,9,4,Marrero,Louisiana
3,"September 4, 2019","Jacksonville, Florida",0,4,4,Three adults and a child were wounded in a sho...,2019-09-04,0,0,2019,9,4,Jacksonville,Florida
4,"September 2, 2019","Elkmont, Alabama",5,0,5,A 14-year-old boy shot dead five family member...,2019-09-02,0,0,2019,9,2,Elkmont,Alabama


Finally save the DataFrame

In [10]:
df.to_csv('Shootings in the USA in 2019.csv')

### Mass Shootings in the USA

In [11]:
url = 'https://en.wikipedia.org/wiki/List_of_mass_shootings_in_the_United_States#Early_2010s'

In [12]:
pageTree = requests.get(url, headers=headers)
soup = BeautifulSoup(pageTree.content, 'html.parser')
tables = soup.find_all("tbody")

# 1 table for each (group of) year(s)
len(tables)

17

In [13]:
menu = soup.find_all('div', {'class': 'toc'})
entries = menu[0].find_all('li', {'class': "toclevel-2"})
years = [entry.find_next('span', {'class': 'toctext'}).text for entry in entries]

column_headers = tables[0].find_all('tr')[0].find_all('th')
col_names = [element.text.strip('\n') for element in column_headers]

In [14]:
dfs = []
for table, year in zip(tables, years):

    data = []
    rows = table.find_all('tr')

    for row in rows[1:]:
        cols = row.find_all('td')
        cols = [element.text for element in cols]
        data.append([element.strip('\n') for element in cols if element]) # Get rid of empty values
        
    df = pd.DataFrame(data, columns = col_names)
    dfs.append(df)
    
df = pd.concat(dfs, axis = 0).reset_index(drop = True)

In [15]:
df['Suspects Dead'] = 0
df['Suspects Dead'] = (df.loc[df['Dead']
                            .str.contains('n \d+'), 'Dead']
                            .str.split('n', expand = True)
                            .iloc[:, 1]
                            .str.replace(']', '')
                            .str.replace('[', '')
                            .str.replace('+', '')
                            .str.strip())
df['Suspects Injured'] = 0
df['Suspects Injured'] = (df.loc[df['Injured']
                            .str.contains('n \d+'), 'Injured']
                            .str.split('n', expand = True)
                            .iloc[:, 1]
                            .str.replace(']', '')
                            .str.replace('[', '')
                            .str.replace('+', '')
                            .str.strip())
                
df['Suspects Dead'] = df['Suspects Dead'].fillna(0)
df['Suspects Injured'] = df['Suspects Injured'].fillna(0)

df['Dead'] = df.Dead.str.replace('+', '')
df['Injured'] = df.Injured.str.replace('+', '')

df['Dead'] = df.Dead.str.replace('n \d+', '', regex = True).str.replace('[]', '', regex = False)
df['Injured'] = df.Injured.str.replace('n \d+', '', regex = True).str.replace('[]', '', regex = False)
df['Injured'] = df.Injured.str.split(' - ', expand = True).fillna(float(np.nan)).astype('float').mean(axis = 1).astype('int')
df['Total'] = df.Total.str.replace('+', '').str.split(' - ', expand = True).fillna(float(np.nan)).astype('float').mean(axis = 1).astype('int')

df[['Dead', 'Injured', 'Total', 'Suspects Dead', 'Suspects Injured']] =\
        df[['Dead', 'Injured', 'Total', 'Suspects Dead', 'Suspects Injured']].astype('int')

In [16]:
##### Examples of problematic datetimes we need to convert (to first day of the range)
# July 23-24, 1968  
# December 31, 1972 - January 7, 1973 

split = df.loc[df.Date.str.contains('[-–]'), 'Date'].str.replace('\n', '').str.split('[-–,]', expand = True)

split_dates = split.iloc[:, :2]

split_dates.loc[split.iloc[:, 1].str.strip().str.len() == 4, 1] =\
                split_dates.loc[split.iloc[:, 1].str.strip().str.len() == 4, 1].values

split_dates.loc[split.iloc[:, 1].str.strip().str.len() != 4, 1] =\
                split.loc[split.iloc[:, 1].str.strip().str.len() != 4, 2].values

df.dt = df.Date
df.loc[df.Date.str.contains('[-–]'), 'dt'] = split_dates[0].str.strip() + ',' + split_dates[1]
df['dt'] = pd.to_datetime(df['dt'].str.strip(), format = '%B %d, %Y')

df['Year'] = df['dt'].apply(lambda x: x.year)
df['Month'] = df['dt'].apply(lambda x: x.month)
df['Day'] = df['dt'].apply(lambda x: x.day)

df['City'] = df.Location.apply(lambda x: x.split(',')[0])
df['State'] = df.Location.apply(lambda x: x.split(',')[-1])

In [17]:
df.head()

,Date,Location,Dead,Injured,Total,Description,Suspects Dead,Suspects Injured,dt,Year,Month,Day,City,State
0,"August 31, 2019","Midland–Odessa, Texas",8,25,33,2019 West Texas shooting: Seven people were ki...,1,0,NaT,NaN,NaN,NaN,Midland–Odessa,Texas
1,"August 4, 2019","Dayton, Ohio",10,27,37,2019 Dayton shooting: Nine people were killed ...,1,0,NaT,NaN,NaN,NaN,Dayton,Ohio
2,"August 3, 2019","El Paso, Texas",22,24,46,2019 El Paso shooting: A gunman killed twenty-...,0,0,NaT,NaN,NaN,NaN,El Paso,Texas
3,"July 28, 2019","Gilroy, California",4,15,19,Gilroy Garlic Festival shooting: A gunman kill...,1,0,NaT,NaN,NaN,NaN,Gilroy,California
4,"May 31, 2019","Virginia Beach, Virginia",13,5,18,Virginia Beach shooting: A gunman killed 12 pe...,1,0,NaT,NaN,NaN,NaN,Virginia Beach,Virginia


In [18]:
df.to_csv('Historical Mass Shootings in the USA.csv')

### School Schootings in the USA

In [19]:
url = 'https://en.wikipedia.org/wiki/List_of_school_shootings_in_the_United_States'

In [20]:
pageTree = requests.get(url, headers=headers)
soup = BeautifulSoup(pageTree.content, 'html.parser')
tables = soup.find_all("tbody")

# 1 table for each (group of) year(s)
len(tables)

20

In [21]:
menu = soup.find_all('div', {'class': 'toc'})
entries = menu[0].find_all('li', {'class': "toclevel-2"})
years = [entry.find_next('span', {'class': 'toctext'}).text for entry in entries]

column_headers = tables[0].find_all('tr')[0].find_all('th')
col_names = [element.text.strip('\n') for element in column_headers]

In [22]:
dfs = []
for table, year in zip(tables, years):

    data = []
    rows = table.find_all('tr')

    for row in rows[1:]:
        cols = row.find_all('td')
        cols = [element.text for element in cols]
        data.append([element.strip('\n') for element in cols if element]) # Get rid of empty values
        
    df = pd.DataFrame(data, columns = col_names)
    dfs.append(df)
    
df = pd.concat(dfs, axis = 0).reset_index(drop = True)

In [23]:
df.shape

(518, 5)

In [24]:
df.head()

,Date,Location,Deaths,Injuries,Description
0,"November 12, 1840","Charlottesville, Virginia",1,0,"John Anthony Gardner Davis, a law professor at..."
1,"November 2, 1853","Louisville, Kentucky",1,0,Student Mathews Flounoy Ward shot the schoolma...
2,"August 16, 1856","Florence, Alabama",2,0,The schoolmaster warned students not to harm h...
3,"July 8, 1858","Baltimore, Maryland",1,0,The 15-year-old son of Col. John T. Farlow (Ba...
4,"January 21, 1860","Todd County, Kentucky",1,0,A son of Col. Elijah Sebree was killed by anot...


In [25]:
df['Suspects Deaths'] = 0
df['Suspects Deaths'] = (df.loc[df['Deaths']
                            .str.contains('n \d+'), 'Deaths']
                            .str.split('n', expand = True)
                            .iloc[:, 1]
                            .str.replace(']', '')
                            .str.replace('[', '')
                            .str.replace('+', '')
                            .str.strip())
df['Suspects Injuries'] = 0
df['Suspects Injuries'] = (df.loc[df['Injuries']
                            .str.contains('n \d+'), 'Injuries']
                            .str.split('n', expand = True)
                            .iloc[:, 1]
                            .str.replace(']', '')
                            .str.replace('[', '')
                            .str.replace('+', '')
                            .str.strip())
                
df['Suspects Deaths'] = df['Suspects Deaths'].fillna(0)
df['Suspects Injuries'] = df['Suspects Injuries'].fillna(0)

df['Deaths'] = df.Deaths.str.replace('+', '')
df['Injuries'] = df.Injuries.str.replace('+', '').str.replace('?', '0')

df['Deaths'] = df.Deaths.str.replace('n \d+', '', regex = True).str.replace('[]', '', regex = False).astype('int')
df['Injuries'] = df.Injuries.str.replace('n \d+', '', regex = True).str.replace('[]', '', regex = False)
df['Injuries'] = df.Injuries.str.split(' - ', expand = True).fillna(float(np.nan)).astype('float').mean(axis = 1).astype('int')
df['Total'] = df.Injuries + df.Deaths


df[['Deaths', 'Injuries', 'Total', 'Suspects Deaths', 'Suspects Injuries']] =\
        df[['Deaths', 'Injuries', 'Total', 'Suspects Deaths', 'Suspects Injuries']].astype('int')

df['dt'] = pd.to_datetime(df['Date'].str.strip(), format = '%B %d, %Y')

In [26]:
df.to_csv('School Shootings in the USA.csv')